Preparing an environment, importing required libs

In [1]:
!pip install "tensorflow-gpu>=1.15.2,<2.0" ampligraph

  Using cached numpy-1.18.5-cp37-cp37m-manylinux1_x86_64.whl (20.1 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.21.6
ERROR: Cannot uninstall numpy 1.21.6, RECORD file not found. You might be able to recover from this via: 'pip install --force-reinstall --no-deps numpy==1.21.6'.
You should consider upgrading via the '/home/dell/f1-knowledge-base/F1-knowledge-base/graph-embeddings/venv/bin/python -m pip install --upgrade pip' command.


In [2]:
import tensorflow as tf
import numpy as np
import pandas as pd
import ampligraph

print(tf.version.VERSION)
ampligraph.__version__

/home/dell/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


1.15.5


'1.4.0'

At first, we have to create pandas dataframe from triples extracted from ontology populated with individuals


In [3]:
from rdflib import Graph, URIRef


ONTOLOGY_IRI = "https://github.com/RogoGit/F1-knowledge-base/f1-ontology"
ONTOLOGY_PREFIX = "f1"
POPULATED_ONTOLOGY_PATH  = '../ontology-with-individuals.owl'


f1_graph = Graph().parse(POPULATED_ONTOLOGY_PATH, format="turtle")
triples_list = []

for subject, predicate, triple_object in f1_graph.triples((None, None, None)):
    if predicate.startswith(URIRef(ONTOLOGY_IRI)):
        triples_list.append([ent.replace(ONTOLOGY_IRI + "#", ONTOLOGY_PREFIX + ":") for ent in [subject, predicate, triple_object]])

f1_df = pd.DataFrame(triples_list, columns = ['Subject', 'Predicate', 'Object'])
print(f1_df)


                                                  Subject  \
0       f1:qualifying_result_2005_turkish_grand_prix_d...   
1       f1:race_result_2005_turkish_grand_prix_fisichella   
2        f1:race_result_1982_brazilian_grand_prix_paletti   
3                                    f1:driver_jacky_ickx   
4         f1:race_result_1976_swedish_grand_prix_nelleman   
...                                                   ...   
362467    f1:race_result_2008_australian_grand_prix_sutil   
362468  f1:race_result_1993_hungarian_grand_prix_blundell   
362469                  f1:death_accident_harry_blanchard   
362470        f1:race_result_1998_german_grand_prix_alesi   
362471      f1:race_result_1990_british_grand_prix_suzuki   

                            Predicate  \
0                     f1:driverNumber   
1                             f1:grid   
2       f1:grandPrixResultIsRelatedTo   
3         f1:hasDriverGrandPrixResult   
4                             f1:grid   
...                  

Next step is to create train and test samples for graph embedding training

In [4]:
from ampligraph.evaluation import train_test_split_no_unseen 

X_train, X_test = train_test_split_no_unseen(np.array(triples_list), test_size=0.10, seed=0)

print('Train set size: ', X_train.shape)
print('Test set size: ', X_test.shape)

Train set size:  (326225, 3)
Test set size:  (36247, 3)


Now it is time to define ComplEx model and train model with train sample

In [6]:
import tensorflow.contrib
from ampligraph.latent_features import ComplEx, save_model

model = ComplEx(batches_count=100,
                epochs=300,
                k=100,
                eta=20,
                optimizer='adam',
                optimizer_params={'lr':1e-4},
                loss='multiclass_nll',
                regularizer='LP',
                regularizer_params={'p':3, 'lambda':1e-5},
                seed=0,
                verbose=True)

model.fit(X_train)
save_model(model, './embedding_model.pkl')

Average ComplEx Loss:   0.110975: 100%|██████████| 300/300 [3:23:58<00:00, 40.79s/epoch]  


In order to use already trained model we can run:

In [ ]:
from ampligraph.latent_features import restore_model

model = restore_model('./embedding_model.pkl')

Next step is to ensure the model can be trained and evaluated correctly. The first of these is defining the filter that will be used to ensure that no negative statements generated by the corruption procedure are actually positives.

In [7]:
from ampligraph.evaluation import evaluate_performance

filter_triples = np.concatenate((X_train, X_test))
ranks = evaluate_performance(X_test,
                             model=model,
                             filter_triples=filter_triples,
                             use_default_protocol=True,
                             verbose=True)

WARNING - DeprecationWarning: use_default_protocol will be removed in future. Please use corrupt_side argument instead.
WARNING - You are attempting to use 76775 distinct entities to generate synthetic negatives in the evaluation
    protocol. This may be unnecessary and will lead to a 'harder' task. Besides, it will lead to a much slower
    evaluation procedure. We recommended to set the 'corruption_entities' argument to a reasonably sized set
    of entities. The size of corruption_entities depends on your domain-specific task.


/home/dell/anaconda3/lib/python3.6/site-packages/ampligraph/evaluation/protocol.py:952: UserWarning: You are attempting to use 76775 distinct entities to generate synthetic negatives in the evaluation
    protocol. This may be unnecessary and will lead to a 'harder' task. Besides, it will lead to a much slower
    evaluation procedure. We recommended to set the 'corruption_entities' argument to a reasonably sized set
    of entities. The size of corruption_entities depends on your domain-specific task.
  warnings.warn(warn_msg % ent_for_corruption_size)
 20%|█▉        | 7069/36247 [26:13<1:48:13,  4.49it/s]


KeyboardInterrupt: 